In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
limit = 5

In [2]:
spark = SparkSession.builder.master("local[2]").appName("Load source").getOrCreate()

In [3]:
spark

#### Postgres Driver Configuration
Note: postgresql-42.7.0.jar require in spark jar forlder

In [4]:
url = "jdbc:postgresql://localhost:5432/glue_db"
format = "jdbc"
driver = "org.postgresql.Driver"
user = "postgres"
password = "khan102030"

In [5]:
filepath = '../../data/hudsonu-product/'

### Read customer data

In [93]:
customer = spark.read.format("csv").\
options(path=filepath + 'Customer.csv',header=True,inferSchema = True,delimiter = ",").load().repartition(3)
# df = spark.read.csv(filepath + 'Customer.csv', header=True).repartition(3)
customer.show(limit)

+-----------+----------+---------+--------------------+--------------+-----+-----+--------------------+------------+
|customer_id|first_name|last_name|             address|          city|state|  zip|               email|       phone|
+-----------+----------+---------+--------------------+--------------+-----+-----+--------------------+------------+
|      49983|  Casandra|   Jarvis|     2 Forster Court|Virginia Beach|   VA|23471|cjarvis6h@nature.com|757-301-5085|
|      13518|   Jilleen|  Downage|02047 Holmberg Ci...|      Waterloo|   IA|50706|  jdownage22@nps.gov|319-104-1110|
|      56390|     Codie| Schrader|  6365 Nelson Street|       Houston|   TX|77245|cschrader7y@seesa...|713-814-7694|
|      56705|  Stillman|Colegrove|87 Independence P...|        Denver|   CO|80243|scolegrovegn@nyma...|303-584-3866|
|      11489|  Jillayne|      Aim| 77998 Vermont Trail|    Santa Cruz|   CA|95064|      jaimr6@nba.com|831-594-6353|
+-----------+----------+---------+--------------------+---------

In [96]:
customer.rdd.getNumPartitions()

3

### Write customer data to postgres

In [97]:
customer.select("*").write.format(format).option("url", url).\
option("driver", driver).option("dbtable", 'customer').\
option("user", user).option("password", password).save()


In [13]:
### Read NetSale data

In [11]:
netsale = spark.read.format("csv").\
options(path=filepath + 'Netsale.csv',header=True,inferSchema = True,delimiter = ",").load()
netsale.show(limit)

+----------+---------------+-----------+--------+-----+------+
|netsale_id|      sale_time|customer_id|shipping|  tax| total|
+----------+---------------+-----------+--------+-----+------+
|  10531095| 1/1/21 1:03 AM|      13467|   16.36| 4.91|103.07|
|  10531270| 1/1/21 9:19 AM|      19177|     0.0|10.57|194.37|
|  10531551|1/1/21 10:55 AM|      55122|   13.99| 5.07| 89.01|
|  10531593|1/1/21 11:10 AM|      22113|     0.0|14.11|239.81|
|  10531612|1/1/21 11:15 AM|      22793|     5.0| 0.72| 20.17|
+----------+---------------+-----------+--------+-----+------+
only showing top 5 rows



In [19]:
netsale_df = netsale.drop("total")

In [20]:
netsale_df.show(2)

+----------+--------------+-----------+--------+-----+
|netsale_id|     sale_time|customer_id|shipping|  tax|
+----------+--------------+-----------+--------+-----+
|  10531095|1/1/21 1:03 AM|      13467|   16.36| 4.91|
|  10531270|1/1/21 9:19 AM|      19177|     0.0|10.57|
+----------+--------------+-----------+--------+-----+
only showing top 2 rows



### Write netsale to Postgres

In [21]:
# netsale_df.select("*").write.format(format).option("url", url).\
# option("driver", driver).option("dbtable", 'netsale').\
# option("user", user).option("password", password).save()

### Read netsale_line 

In [22]:
netsale_line = spark.read.format("csv").\
options(path=filepath + 'Netsale_Line.csv',header=True,inferSchema = True,delimiter = ",").load()
netsale_line.show(limit)

+----------+----------+-----+---+-----+
|netsale_id|product_id|price|qty| size|
+----------+----------+-----+---+-----+
|  10531095|  75361262|14.55|  3| NULL|
|  10531095|  78150032|10.85|  3| NULL|
|  10531095|  87404608| 56.4|  7|Large|
|  10533233|  74183131| 27.3|  1| NULL|
|  10533250|  77024694| 10.9|  4| NULL|
+----------+----------+-----+---+-----+
only showing top 5 rows



### Write netsale_line to Postgres

In [23]:
# netsale_line.select("*").write.format(format).option("url", url).\
# option("driver", driver).option("dbtable", 'netsale_line').\
# option("user", user).option("password", password).save()

In [14]:
# df = netsale_line.join(netsale,netsale_line.netsale_id == netsale.netsale_id,"inner").\
# select(netsale.netsale_id,netsale.customer_id,netsale.sale_time,
#        netsale.shipping,netsale.tax,netsale.total,
#        netsale_line.product_id,
#        netsale_line.price,netsale_line.qty)

### Read Product

In [30]:
product = spark.read.format("csv").\
options(path=filepath + 'Product.csv',header=True,inferSchema = True,delimiter = ",").load()
product.show(limit,True)

+----------+--------------------+--------+------------------+--------------------+----------+--------------------+--------+--------------------+--------------------+-----------+------------------------+--------------------+-----------+----------------+-------------+
|product_id|                name|in_store|embellishment_type|              fabric|       fit|        graphic_type|  origin|         season_code|              vendor| color_code|single_sku_catalog_entry|             catalog|   category|     subcategory|current_price|
+----------+--------------------+--------+------------------+--------------------+----------+--------------------+--------+--------------------+--------------------+-----------+------------------------+--------------------+-----------+----------------+-------------+
|  13029606|White ^un^ Coache...|    true|  Embroidered/Sewn|97% Polyester/3% ...|      NULL|            SIDELINE|Imported|                Fall|              Adidas|      White|               1030000

In [60]:
product.columns[:5]

['product_id', 'name', 'in_store', 'embellishment_type', 'fabric']

In [62]:
product_df =product.withColumn('discount',FS.lit(10)).\
withColumnRenamed('current_price',"mrp").\
withColumn('sale_price',(FS.col('mrp') * (100 - FS.col('discount'))) /100)

In [65]:
product_df.columns[:6]

['product_id', 'name', 'in_store', 'embellishment_type', 'fabric', 'fit']

### Write Product to Postgres

In [66]:
# product_df.select("*").write.format(format).option("url", url).\
# option("driver", driver).option("dbtable", 'product').\
# option("user", user).option("password", password).save()

### Read Size

In [67]:
size_df = spark.read.format("csv").\
options(path=filepath + 'Size.csv',header=True,inferSchema = True,delimiter = ",").load()
size_df.show(limit,True)

+----------+-------+
|product_id|   size|
+----------+-------+
|  57450359|2XLarge|
|  57450359|  Large|
|  57450359| Medium|
|  57450359|  Small|
|  57450359| XLarge|
+----------+-------+
only showing top 5 rows



### Write size to Postgres

In [69]:
# size_df.select("*").write.format(format).option("url", url).\
# option("driver", driver).option("dbtable", 'product_size').\
# option("user", user).option("password", password).save()

### Read Vendor

In [72]:
vendor = spark.read.format("csv").\
options(path=filepath + 'Vendor.csv',header=True,inferSchema = True,delimiter = ",").load()
vendor.show(limit,True)

+--------------------+--------------------+--------------------+--------+-----------+-----+-------+--------------+--------------+
|                name|                 url|             address|address2|       city|state|    zip|         phone|           fax|
+--------------------+--------------------+--------------------+--------+-----------+-----+-------+--------------+--------------+
|                  47|https://www.47bra...|132 Campanelli In...| Door 27|   Brockton|   MA| 2301.0|  781-702-2921|          NULL|
|              Adidas|https://www.adida...|5055 N Greeley Av...|    NULL|   Portland|   OR|97217.0|(800) 982-9337|(971) 234-2450|
|    Alex And Ani Inc|https://www.alexa...|2000 Chapel View ...|    NULL|   Cranston|   RI| 2920.0|  800-725-7822|          NULL|
|       All Star Dogs|http://www.allsta...|    20 Hastings Road|    NULL|   Marlboro|   NJ| 7746.0|  800-391-3520|  732-566-3401|
|Alta Gracia Holdc...|http://www.altagr...| 160 E Saint John St|    NULL|Spartanburg|   SC

### Write Vendor to Postgres

In [73]:
# vendor.select("*").write.format(format).option("url", url).\
# option("driver", driver).option("dbtable", 'vendor').\
# option("user", user).option("password", password).save()

### Read Wholesale

In [74]:
wholesale = spark.read.format("csv").\
options(path=filepath + 'Wholesale.csv',header=True,inferSchema = True,delimiter = ",").load()
wholesale.show(limit,True)

+--------+-----------+-------------------+
|order_id|     outlet|               date|
+--------+-----------+-------------------+
|   80001|     K-Mart|2010-01-02 00:00:00|
|   80002|     Target|2010-01-02 00:00:00|
|   80003|Sam 's Club|2010-01-03 00:00:00|
|   80004|     Target|2010-01-04 00:00:00|
|   80005|    Walmart|2010-01-04 00:00:00|
+--------+-----------+-------------------+
only showing top 5 rows



### Write Wholesale to Postgres

In [75]:
wholesale.select("*").write.format(format).option("url", url).\
option("driver", driver).option("dbtable", 'wholesale').\
option("user", user).option("password", password).save()

### Read Wholesale_Line

In [76]:
wholesale_line = spark.read.format("csv").\
options(path=filepath + 'Wholesale_Line.csv',header=True,inferSchema = True,delimiter = ",").load()
wholesale_line.show(limit,True)

+--------+----------+-------+-----+--------+
|order_id|product_id|   size|price|quantity|
+--------+----------+-------+-----+--------+
|   80065|  94402140|  Small| 25.5|     262|
|   80065|  94402140| Medium| 25.5|     210|
|   80065|  94402140|  Large| 25.5|     221|
|   80065|  94402140|3XLarge| 25.5|     227|
|   80065|  94402140|2XLarge| 25.5|     230|
+--------+----------+-------+-----+--------+
only showing top 5 rows



### Write Wholesale_Line to Postgres

In [77]:
wholesale_line.select("*").write.format(format).option("url", url).\
option("driver", driver).option("dbtable", 'wholesale_line').\
option("user", user).option("password", password).save()